## 1. Load the packages

In [1]:
# Data processing packages
import numpy as np
import pandas as pd
from collections import Counter

# Machine learning packages
from sklearn.preprocessing import MinMaxScaler, StandardScaler, MultiLabelBinarizer, FunctionTransformer
from sklearn.pipeline import Pipeline
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.neighbors import KNeighborsRegressor
import torch

# Visualization packages
import seaborn as sns
import matplotlib.pyplot as plt

# Others
import time

## 2. Read the data

In [2]:
X1 = torch.load('X1_ready')
Y1 = pd.read_csv("Y1.csv", header=None, names=['revenue ']).values

## 4. Feature Selection

### 4.1 Forward Feature Selection

In [ ]:
knnr = KNeighborsRegressor(n_jobs=-1)
foward_feature_selection = SFS(
    knnr,
    k_features=10,
    forward=True,
    floating=False,
    verbose=2,
    # scoring='accuracy',
    cv=5
).fit(X1, Y1)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
